
# Bar Inventory Analysis & Forecasting 📊🍸

**Dataset:** Consumption Dataset - Bar Inventory  
**Objective:** Exploratory Data Analysis (EDA), Forecasting (ARIMA, Prophet), Inventory Simulation (Par Level, Reorder Point), Reporting.

---

**Sections:**  
✅ Descriptive Statistics and EDA  
✅ Visualization (Trends, Outliers, Consumption)  
✅ Forecasting (Moving Averages, ARIMA, Prophet)  
✅ Inventory Recommendation (Par Level, Reorder Point)  
✅ Simulation (Inventory scenarios)  
✅ Reporting Insights


In [ ]:

# Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')

# For timeseries
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [ ]:

# Load Dataset
df = pd.read_csv('/content/Consumption Dataset - Dataset.csv')
df.head()


In [ ]:

# Convert Date column
df['Date'] = pd.to_datetime(df['Date Time Served'].str.split(' ').str[0], format='%m/%d/%Y')
df = df.sort_values('Date')

# Basic Info
df.info()


### Descriptive Statistics

In [ ]:

df.describe()[['Opening Balance (ml)', 'Purchase (ml)', 'Consumed (ml)', 'Closing Balance (ml)']]


### Total Daily Consumption Trend

In [ ]:

daily_consumption = df.groupby('Date')['Consumed (ml)'].sum()
daily_consumption.plot(figsize=(14,6), title='Daily Total Consumption (ml)', color='teal')
plt.ylabel('Consumed (ml)')
plt.xlabel('Date')
plt.grid(True)
plt.show()


### Top 10 Brands by Total Consumption

In [ ]:

top_brands = df.groupby('Brand Name')['Consumed (ml)'].sum().sort_values(ascending=False).head(10)
sns.barplot(x=top_brands.values, y=top_brands.index, palette='viridis')
plt.title('Top 10 Brands by Total Consumption')
plt.xlabel('Consumed (ml)')
plt.show()


### Consumption by Bar Location

In [ ]:

bar_consumption = df.groupby('Bar Name')['Consumed (ml)'].sum().sort_values(ascending=False)
sns.barplot(x=bar_consumption.values, y=bar_consumption.index, palette='Set2')
plt.title('Total Consumption by Bar')
plt.xlabel('Consumed (ml)')
plt.show()


### Outliers in Consumption by Alcohol Type

In [ ]:

sns.boxplot(x='Alcohol Type', y='Consumed (ml)', data=df, palette='coolwarm')
plt.title('Outliers in Consumption by Alcohol Type')
plt.show()


### Moving Average & Decomposition

In [ ]:

# 7-day moving average
daily_consumption_ma7 = daily_consumption.rolling(7).mean()

plt.figure(figsize=(14,6))
plt.plot(daily_consumption, label='Daily Consumption')
plt.plot(daily_consumption_ma7, label='7-day Moving Avg', linewidth=3)
plt.title('Consumption Trend with Moving Average')
plt.legend()
plt.grid(True)
plt.show()

# Seasonal Decomposition
decomposition = seasonal_decompose(daily_consumption, model='additive', period=7)
decomposition.plot()
plt.show()


### ARIMA Forecasting

In [ ]:

# Simple ARIMA Model (SARIMA seasonal)
sarima_model = SARIMAX(daily_consumption, order=(1,1,1), seasonal_order=(1,1,1,7))
sarima_result = sarima_model.fit(disp=False)
sarima_forecast = sarima_result.get_forecast(steps=30)
sarima_ci = sarima_forecast.conf_int()

# Plot
plt.figure(figsize=(14,6))
plt.plot(daily_consumption, label='Observed')
plt.plot(sarima_forecast.predicted_mean, label='SARIMA Forecast', color='red')
plt.fill_between(sarima_ci.index, sarima_ci.iloc[:, 0], sarima_ci.iloc[:, 1], color='pink', alpha=0.3)
plt.title('SARIMA Forecast for Next 30 Days')
plt.legend()
plt.grid(True)
plt.show()


### Prophet Forecasting

In [ ]:

# Prepare data
prophet_df = daily_consumption.reset_index()
prophet_df.columns = ['ds', 'y']

# Prophet model
prophet_model = Prophet()
prophet_model.fit(prophet_df)

# Future
future = prophet_model.make_future_dataframe(periods=30)
forecast = prophet_model.predict(future)

# Plot
fig1 = prophet_model.plot(forecast)
plt.title('Prophet Forecast: 30 Days')
plt.show()

# Components
fig2 = prophet_model.plot_components(forecast)
plt.show()


### Inventory Recommendation (Par Level, Safety Stock, Reorder Point)

In [ ]:

# Define functions
def calculate_safety_stock(std_dev, z_score=1.65):  # 95% service level
    return std_dev * z_score

def calculate_reorder_point(avg_demand, lead_time, safety_stock):
    return (avg_demand * lead_time) + safety_stock

# Example: Top brand
brand_example = 'Captain Morgan'
df_brand = df[df['Brand Name'] == brand_example]
daily_brand = df_brand.groupby('Date')['Consumed (ml)'].sum()

# Calculate stats
avg_daily = daily_brand.mean()
std_daily = daily_brand.std()
lead_time_days = 7

# Inventory metrics
safety_stock = calculate_safety_stock(std_daily)
reorder_point = calculate_reorder_point(avg_daily, lead_time_days, safety_stock)

print(f"Brand: {brand_example}")
print(f"Avg Daily Demand: {avg_daily:.2f} ml")
print(f"Safety Stock: {safety_stock:.2f} ml")
print(f"Reorder Point: {reorder_point:.2f} ml")


### Inventory Simulation

In [ ]:

# Simulate stock level
stock_level = 5000  # initial
sim_days = 30
daily_demand_sim = np.random.normal(avg_daily, std_daily, sim_days)

stock_trace = []
for day in range(sim_days):
    stock_level -= daily_demand_sim[day]
    if stock_level < reorder_point:
        stock_level += 7000  # assume replenishment quantity
    stock_trace.append(stock_level)

# Plot
plt.figure(figsize=(14,6))
plt.plot(stock_trace, marker='o')
plt.axhline(reorder_point, color='red', linestyle='--', label='Reorder Point')
plt.title(f'Inventory Simulation for {brand_example} - 30 Days')
plt.ylabel('Stock Level (ml)')
plt.xlabel('Day')
plt.legend()
plt.grid(True)
plt.show()


### Final Insights & Recommendations


- High-demand brands: Adjust par levels and safety stock.
- Bars with frequent stockouts: Optimize reorder points.
- Time series forecasts (ARIMA & Prophet) show clear seasonal patterns.
- Recommended next steps:
    - Automate this pipeline weekly.
    - Train on more brands individually.
    - Implement dynamic safety stock logic based on actual lead times.
